In [7]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np
import heapq
import copy

In [8]:
class InfiniteGrid(collections.defaultdict):
    def __init__(self, default='.'):
        super().__init__(lambda: collections.defaultdict(lambda: default))
        self.dimensions_x = None
        self.dimensions_y = None
        
    def __setitem__(self, pos, val):
        if type(pos) is int:
            return super().__setitem__(pos, val)
        
        yy, xx = pos

        if self.dimensions_x is None:
            self.dimensions_x = (xx, xx)
        else:
            self.dimensions_x = (
                min(xx, self.dimensions_x[0]),
                max(xx, self.dimensions_x[1])
            )
        
        if self.dimensions_y is None:
            self.dimensions_y = (yy, yy)
        else:
            self.dimensions_y = (
                min(yy, self.dimensions_y[0]),
                max(yy, self.dimensions_y[1])
            )    
            

        super().__getitem__(yy)[xx] = val

    def __getitem__(self, pos):
        if type(pos) is int:
            return super().__getitem__(pos)

        yy, xx = pos
        return super().__getitem__(yy)[xx]

    def iterrow(self, row):
        row_data = self[row]
        for xx in range(self.dimensions_x[0], self.dimensions_x[1]+1):
            yield row_data[xx]

    def enumeraterow(self, row):
        row_data = self[row]
        for xx in range(self.dimensions_x[0], self.dimensions_x[1]+1):
            yield (row, xx), row_data[xx]

    def __str__(self):
        if self.dimensions_x is None or self.dimensions_y is None:
            return ""

        res = []
        for yy in range(self.dimensions_y[0], self.dimensions_y[1]+1):
            res.append("%02d " % yy)
            for xx in range(self.dimensions_x[0], self.dimensions_x[1]+1):
                 res.append(str(self[yy][xx]))
            res.append("\n")

        return "".join(res)  


In [9]:
def read_input(puzzle_input):
    with open(puzzle_input) as ff:
        dd = ff.readlines()

    field = InfiniteGrid('.')
    
    cmds = [ cmd.strip() for cmd in dd[0] ]
    
    return field, cmds

def test_read_input():
    field, cmds = read_input(os.path.join(os.path.join("..", "dat", "day15_test.txt")))
    assert(len(cmds)) == 40
    assert(cmd[0]) == '>'
    assert(cmd[4]) == '<'
    assert(cmd[-1]) == '>'
    assert(cmd[-3]) == '<'

In [10]:
def create_rocks():
    rocks = [
        np.array([(1,1,1,1), (0,0,0,0), (0,0,0,0), (0,0,0,0)]),
        np.array([(0,1,0,0), (1,1,1,0), (0,1,0,0), (0,0,0,0)]),
        np.array([(1,1,1,0), (0,0,1,0), (0,0,1,0), (0,0,0,0)]),
        np.array([(1,0,0,0), (1,0,0,0), (1,0,0,0), (1,0,0,0)]),
        np.array([(1,1,0,0), (1,1,0,0), (0,0,0,0), (0,0,0,0)])
    ]

    return rocks

def test_rocks():
    for rock in create_rocks():
        for row in range(3, -1, -1):
            for col in range(4):
                print(rock[row][col], end="")
            print("")
        print("")

test_rocks()

0000
0000
0000
1111

0000
0100
1110
0100

0000
0010
0010
1110

1000
1000
1000
1000

0000
0000
1100
1100



In [89]:
rock_width = [ 4, 3, 3, 1, 2 ]

def check_collision(field, rock, pos):
    for row in range(4):
        for col in range(4):
            if rock[row, col] == 1 and field[pos[0]+row, pos[1]+col] != '.':
                return True

    return False

def print_field(field, height, rock=None, rock_pos=None):
    for row in range(height, 0, -1):
        for col in range(0, 7):
            if rock is not None:
                rock_row = row-rock_pos[0]
                rock_col = col-rock_pos[1]
                if 0 <= rock_row < 4 and 0 <= rock_col < 4 and rock[rock_row][rock_col] == 1:
                    print('@', end="")
                else:
                    print(field[row, col], end="")
            else:
                print(field[row, col], end="")
        print("")
    print("-"*7)
    print()

def part1(puzzle_input):
    field, cmds = read_input(puzzle_input)

    height = 0

    # rocks are positioned relative to the left-bottom edge of the rock being (0,0) (yy, xx)
    rocks = create_rocks()
    rock_ii = 0
    cmd_ii = 0
    step = 0
    for _ in range(2022):
        rock = rocks[rock_ii]
        pos = (height+4, 2)

        #print("*"*40)
        #print_field(field, height+10, rock, pos)
            
        while True:
            cmd = cmds[cmd_ii]
            cmd_ii = (cmd_ii + 1) % len(cmds)
            
            if cmd == '>':
                next_pos = pos[0], pos[1] + 1
            elif cmd == '<':
                next_pos = pos[0], pos[1] - 1

            if next_pos[1] < 0 or (next_pos[1] + rock_width[rock_ii]) > 7:
                next_pos = pos

            if check_collision(field, rock, next_pos):
                next_pos = pos
            else:
                pos = next_pos

            step += 1
            #print("Step", step)
            #print("cmd", cmd)
            #print_field(field, height+10, rock, pos)
            
            next_pos = pos[0] - 1, pos[1]

            if check_collision(field, rock, next_pos) or next_pos[0] == 0:
                for row in range(4):
                    for col in range(4):
                        if rock[row][col] == 1:
                            field[pos[0]+row][pos[1]+col] = "#"
                            height = max(height, pos[0]+row)
                
                #print(height)
                #print_field(field, height+10, rock, pos)
                break

            # Drop one
            #print("drop")
            #print_field(field, height+10, rock, next_pos)
            pos = next_pos
        
        rock_ii = (rock_ii + 1) % len(rocks)    

    return height

def test_part1():
    ans = part1(os.path.join(os.path.join("..", "dat", "day17_test.txt")))
    assert ans == 3068

test_part1()

ans = part1(os.path.join(os.path.join("..", "dat", "day17.txt")))
print(ans)


3067


In [16]:
rock_width = [ 4, 3, 3, 1, 2 ]

def check_collision(field, rock, pos):
    for row in range(4):
        for col in range(4):
            if rock[row, col] == 1 and field[pos[0]+row, pos[1]+col] != '.':
                return True

    return False

def print_field(field, height, rock=None, rock_pos=None):
    for row in range(height, 0, -1):
        for col in range(0, 7):
            if rock is not None:
                rock_row = row-rock_pos[0]
                rock_col = col-rock_pos[1]
                if 0 <= rock_row < 4 and 0 <= rock_col < 4 and rock[rock_row][rock_col] == 1:
                    print('@', end="")
                else:
                    print(field[row, col], end="")
            else:
                print(field[row, col], end="")
        print("")
    print("-"*7)
    print()

def part2(puzzle_input):
    field, cmds = read_input(puzzle_input)

    height = 0
    last_height = 0
    last_xx = 0
    # rocks are positioned relative to the left-bottom edge of the rock being (0,0) (yy, xx)
    rocks = create_rocks()
    rock_ii = 0
    cmd_ii = 0
    step = 0
    first = False
    height_offset = 0

    rocks_remaining = 1000000000000
    xx = 0
    cycle_height = None
    seen = {}
    while rocks_remaining > 0:
        xx += 1
        rocks_remaining -= 1
        rock = rocks[rock_ii]
        pos = (height+4, 2)

        #print("*"*40)
        #print_field(field, height+10, rock, pos)
            
        start_cmd_ii = cmd_ii
        while True:
            cmd = cmds[cmd_ii]
            cmd_ii = (cmd_ii + 1) % len(cmds)
            
            if cmd == '>':
                next_pos = pos[0], pos[1] + 1
            elif cmd == '<':
                next_pos = pos[0], pos[1] - 1

            if next_pos[1] < 0 or (next_pos[1] + rock_width[rock_ii]) > 7:
                next_pos = pos

            if check_collision(field, rock, next_pos):
                next_pos = pos
            else:
                pos = next_pos

            step += 1
            #print("Step", step)
            #print("cmd", cmd)
            #print_field(field, height+10, rock, pos)
            
            next_pos = pos[0] - 1, pos[1]

            if check_collision(field, rock, next_pos) or next_pos[0] == 0:
                for row in range(4):
                    for col in range(4):
                        if rock[row][col] == 1:
                            field[pos[0]+row][pos[1]+col] = "#"
                            height = max(height, pos[0]+row)
                
                #print(height)
                #print_field(field, height+10, rock, pos)
                break

            # Drop one
            #print("drop")
            #print_field(field, height+10, rock, next_pos)
            pos = next_pos
        
        # Was there a command rollover
        # Look for repeat on the jet cycle where the same rock index and cycle index occur a second time
        # this is likely a simplification that works for AoC but wouldn't work broadly...a proper solution
        # should look for long-cycles in height increments
        if start_cmd_ii > cmd_ii:
            if (cmd_ii, rock_ii) in seen:
                cycle_height = height - seen[cmd_ii, rock_ii][0]
                rocks_used = xx - seen[cmd_ii, rock_ii][1]

                print("Rocks Used %s Cycle Height %s" %(rocks_used, cycle_height))
                height_offset = (rocks_remaining // rocks_used) * cycle_height
                rocks_remaining = (rocks_remaining % rocks_used)
                print("Fast Forward", height+height_offset, rocks_remaining)
            else:
                seen[cmd_ii, rock_ii] = (height, xx)
    
        rock_ii = (rock_ii + 1) % len(rocks)
    print("Height", height, height-last_height, xx+1, xx-last_xx, cmd_ii, rock_ii)
    return height+height_offset

def test_part2():
    ans = part2(os.path.join(os.path.join("..", "dat", "day17_test.txt")))
    assert ans == 1514285714288

test_part2()


ans = part2(os.path.join(os.path.join("..", "dat", "day17.txt")))
print(ans)


Rocks Used 35 Cycle Height 53
Fast Forward 1514285714288 0
Height 78 78 51 50 2 0
Rocks Used 1705 Cycle Height 2582
Fast Forward 1514369499075 1580
Height 7582 7582 4996 4995 9341 0
1514369501484
